# Lab | Making predictions with logistic regression

In this lab, you will be using the [Sakila](https://dev.mysql.com/doc/sakila/en/) database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.


### Instructions

1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.
2. Create a query to get the list of films and a boolean indicating if it was rented last month (May 2005). This would be our target variable.
3. Read the data into a Pandas dataframe.
4. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.
5. Create a logistic regression model to predict this variable from the cleaned data.
6. Evaluate the results.


## Import libraries and get database password

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

## Get database data through sql

In [3]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = ''' select title, rental_duration, rental_rate, length, replacement_cost, rating, category, rented_may
            from(
            select film_id,
            case
            when month(rental_date) = 5 then True
            else False
            end as rented_may
            from rental
            join inventory using(inventory_id)
            join film using(film_id)
            join film_category using(film_id)
            join category using (category_id)
            where year(rental_date) = 2005 and month(rental_date) = 5
            group by film_id) t1
            right join(
            select rental_date, title, film_id, rental_duration, rental_rate, length, replacement_cost, rating, category.name as category
            from rental
            join inventory using(inventory_id)
            join film using(film_id)
            join film_category using(film_id)
            join category using (category_id)
            where year(rental_date) = 2005
            group by film_id
            ) t2
            using (film_id)
            order by title asc'''

data = pd.read_sql_query(query, engine)
data.head(30)

,title,rental_duration,rental_rate,length,replacement_cost,rating,category,rented_may
0,ACADEMY DINOSAUR,6,0.99,86,20.99,PG,Documentary,1.0
1,ACE GOLDFINGER,3,4.99,48,12.99,G,Horror,NaN
2,ADAPTATION HOLES,7,2.99,50,18.99,NC-17,Documentary,1.0
3,AFFAIR PREJUDICE,5,2.99,117,26.99,G,Horror,1.0
4,AFRICAN EGG,6,2.99,130,22.99,G,Family,1.0
5,AGENT TRUMAN,3,2.99,169,17.99,PG,Foreign,1.0
6,AIRPLANE SIERRA,6,4.99,62,28.99,PG-13,Comedy,NaN
7,AIRPORT POLLOCK,6,4.99,54,15.99,R,Horror,1.0
8,ALABAMA DEVIL,3,2.99,114,21.99,PG-13,Horror,NaN
9,ALADDIN CALENDAR,6,4.99,63,24.99,NC-17,Sports,NaN


In [4]:
# should be categorized --- probably gonna drop this one
data['replacement_cost'].value_counts()

20.99    55
21.99    55
22.99    54
29.99    52
12.99    52
27.99    51
13.99    50
14.99    48
11.99    47
17.99    46
10.99    46
26.99    45
19.99    45
23.99    44
25.99    41
9.99     40
28.99    40
18.99    40
24.99    37
16.99    36
15.99    34
Name: replacement_cost, dtype: int64

In [5]:
# should be categorized
data['rental_duration'].value_counts()

6    203
3    197
4    194
5    186
7    178
Name: rental_duration, dtype: int64

In [6]:
# should categorized categorized
data['rental_rate'].value_counts()

0.99    326
4.99    320
2.99    312
Name: rental_rate, dtype: int64

In [7]:
# bining the movies. 0,90,120,180-max
data['length'].value_counts()

85     17
179    13
84     13
112    12
122    11
       ..
94      3
96      2
55      2
66      2
95      2
Name: length, Length: 140, dtype: int64

In [8]:
data.isna().sum()

title                 0
rental_duration       0
rental_rate           0
length                0
replacement_cost      0
rating                0
category              0
rented_may          272
dtype: int64

In [9]:
data.dtypes

title                object
rental_duration       int64
rental_rate         float64
length                int64
replacement_cost    float64
rating               object
category             object
rented_may          float64
dtype: object

In [10]:
data['rented_may'] = data['rented_may'].fillna(0)

In [11]:
data.head()

,title,rental_duration,rental_rate,length,replacement_cost,rating,category,rented_may
0,ACADEMY DINOSAUR,6,0.99,86,20.99,PG,Documentary,1.0
1,ACE GOLDFINGER,3,4.99,48,12.99,G,Horror,0.0
2,ADAPTATION HOLES,7,2.99,50,18.99,NC-17,Documentary,1.0
3,AFFAIR PREJUDICE,5,2.99,117,26.99,G,Horror,1.0
4,AFRICAN EGG,6,2.99,130,22.99,G,Family,1.0


In [12]:
data = data[['title','category', 'rating', 'length','rental_rate','replacement_cost','rental_duration','rented_may']]
data.head()

,title,category,rating,length,rental_rate,replacement_cost,rental_duration,rented_may
0,ACADEMY DINOSAUR,Documentary,PG,86,0.99,20.99,6,1.0
1,ACE GOLDFINGER,Horror,G,48,4.99,12.99,3,0.0
2,ADAPTATION HOLES,Documentary,NC-17,50,2.99,18.99,7,1.0
3,AFFAIR PREJUDICE,Horror,G,117,2.99,26.99,5,1.0
4,AFRICAN EGG,Family,G,130,2.99,22.99,6,1.0


# next steps:
Columns to categorical encode: title, category, rating, rental_rate, rental duration.
Columns to drop: replacement_cost
Columns to bin: length

Target column: rented_may

In [13]:
data = data.drop(['replacement_cost'], axis = 1)
data.head()

# droping column replacement_cost, decided that it's unneccessary as a feature

,title,category,rating,length,rental_rate,rental_duration,rented_may
0,ACADEMY DINOSAUR,Documentary,PG,86,0.99,6,1.0
1,ACE GOLDFINGER,Horror,G,48,4.99,3,0.0
2,ADAPTATION HOLES,Documentary,NC-17,50,2.99,7,1.0
3,AFFAIR PREJUDICE,Horror,G,117,2.99,5,1.0
4,AFRICAN EGG,Family,G,130,2.99,6,1.0


In [14]:
data.dtypes

title               object
category            object
rating              object
length               int64
rental_rate        float64
rental_duration      int64
rented_may         float64
dtype: object

In [15]:
data['rental_rate'] = data['rental_rate'].astype(object)
data['rental_duration'] = data['rental_duration'].astype(object)

# converting columns to object

In [16]:
lbl = ['short', 'normal', 'long', 'extended'] 
data['length'] = pd.cut(data['length'],[0,90,120,150,int(data['length'].max())], labels=lbl) 
data['length'].value_counts()

# making bins for length 0,90,120,150-max

short       311
extended    233
normal      207
long        207
Name: length, dtype: int64

# starting the regression analysis

In [17]:
# X-y Split
X = data.drop(['rented_may'], axis=1)
y = data['rented_may']
    
# Train test Splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [18]:
X_train.head()

,title,category,rating,length,rental_rate,rental_duration
302,FLAMINGOS CONNECTICUT,New,PG-13,short,4.99,4
467,JUMPING WRATH,Children,NC-17,short,0.99,4
294,FICTION CHRISTMAS,Foreign,PG,short,0.99,4
548,MIDNIGHT WESTWARD,Action,G,short,0.99,3
465,JUGGLER HARDLY,Animation,PG-13,short,0.99,4


In [19]:
y_train.head()

302    1.0
467    1.0
294    0.0
548    1.0
465    1.0
Name: rented_may, dtype: float64

In [20]:
# onehotencoder for the categorical data, fitting only for once to train data
onehotencoder = OneHotEncoder(drop='first', handle_unknown = 'ignore'). fit(X_train)


# defining the encoding function here to apply it to the X categorical data
def encode_cats(X_cat):
    encoded = onehotencoder.transform(X_cat).toarray()
    cols = onehotencoder.get_feature_names(input_features=X_cat.columns)
    return pd.DataFrame(encoded, columns=cols)

# all of our X_train data is categorical so applying func to X_train data
X_train = encode_cats(X_train)
X_train.head()

/Users/begumerdem/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,title_ACE GOLDFINGER,title_ADAPTATION HOLES,title_AFFAIR PREJUDICE,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_AIRPORT POLLOCK,title_ALABAMA DEVIL,title_ALADDIN CALENDAR,title_ALAMO VIDEOTAPE,...,rating_R,length_long,length_normal,length_short,rental_rate_2.99,rental_rate_4.99,rental_duration_4,rental_duration_5,rental_duration_6,rental_duration_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [21]:
# onehotencoder for X_test data without fitting again
X_test = encode_cats(X_test)
X_test.head()

/Users/begumerdem/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/begumerdem/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,title_ACE GOLDFINGER,title_ADAPTATION HOLES,title_AFFAIR PREJUDICE,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_AIRPORT POLLOCK,title_ALABAMA DEVIL,title_ALADDIN CALENDAR,title_ALAMO VIDEOTAPE,...,rating_R,length_long,length_normal,length_short,rental_rate_2.99,rental_rate_4.99,rental_duration_4,rental_duration_5,rental_duration_6,rental_duration_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [22]:
X_test.head()

,title_ACE GOLDFINGER,title_ADAPTATION HOLES,title_AFFAIR PREJUDICE,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_AIRPORT POLLOCK,title_ALABAMA DEVIL,title_ALADDIN CALENDAR,title_ALAMO VIDEOTAPE,...,rating_R,length_long,length_normal,length_short,rental_rate_2.99,rental_rate_4.99,rental_duration_4,rental_duration_5,rental_duration_6,rental_duration_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [23]:
# time for regression analysis

from sklearn.linear_model import LogisticRegression

classification = LogisticRegression(solver='saga').fit(X_train, y_train)
predictions = classification.predict(X_test)
classification.score(X_test, y_test)

0.7083333333333334

In [24]:
pd.Series(y_test).value_counts()

1.0    136
0.0     56
Name: rented_may, dtype: int64

In [26]:
pd.Series(predictions).value_counts()
# overchoosing-emphasizing problem here

1.0    184
0.0      8
dtype: int64

In [27]:
# trying KNN classifier as a second model and see if it give more accuracy
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=4, weights='uniform')
clf.fit(X_train, y_train)
predictions_clf = clf.predict(X_test)
clf.score(X_test, y_test)


0.5416666666666666

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions_clf)

array([[ 9, 47],
       [41, 95]])

In [30]:
pd.Series(y_test).value_counts()

1.0    136
0.0     56
Name: rented_may, dtype: int64

In [31]:
pd.Series(predictions_clf).value_counts()
# not sure which model is more accurate KNN or SAGA

1.0    142
0.0     50
dtype: int64